In [ ]:
# hide

%load_ext nb_black
%config InlineBackend.figure_format="retina"

<IPython.core.display.Javascript object>

In [ ]:
# dont_test

import httpx
import pandas as pd

from urllib.parse import urljoin

from will_it_saturate import servers
from will_it_saturate import clients

# from will_it_saturate.gevent_client import GeventClient
from will_it_saturate.epochs import Epoch
from will_it_saturate.hosts import Host, HostDetails

from will_it_saturate.results import Result
from will_it_saturate.files import BenchmarkFile
from will_it_saturate.control.client import ControlClient
from will_it_saturate.repositories import SqliteRepository, register_default_tables

<IPython.core.display.Javascript object>

In [ ]:
# dont_test

server_control_host = Host(name="localhost", port=8100)
server_control_client = ControlClient(host=server_control_host)
server_details = server_control_client.get_host_details()
# server = server_control_client.get_or_create_server(
#     servers.FastAPIUvicornServer(host="localhost", port=5001)
# )
server = server_control_client.get_or_create_server(
    servers.CaddyFileServer(host="localhost", port=5001)
)

<IPython.core.display.Javascript object>

In [ ]:
# dont_test

byte, file_size, duration = 8, 10**5, 3
epoch = Epoch(file_size=file_size, duration=duration)
files = server_control_client.get_or_create_files(epoch)
epoch.urls = [server.file_to_url(file) for file in files]

<IPython.core.display.Javascript object>

In [ ]:
# dont_test

client_control_host = Host(name="localhost", port=8100)
client_control_client = ControlClient(host=client_control_host)
client_details = client_control_client.get_host_details()
# benchmark_client = GeventClient(name="gevent", host=server_host_name, port=server_port)
# benchmark_client = GeventClient(name="gevent")
# benchmark_client = clients.WrkClient(name="wrk")
# benchmark_client = clients.AioHttpClient(name="aiohttp")
benchmark_client = clients.HttpxClient(name="httpx")

<IPython.core.display.Javascript object>

In [ ]:
# dont_test

result = Result(
    server=server,
    client=benchmark_client,
    server_details=server_details,
    client_details=client_details,
    file_size=epoch.file_size,
    complete_size=epoch.complete_size,
)

<IPython.core.display.Javascript object>

In [ ]:
# dont_test

result.dict()

{'server': {'protocol': 'http',
  'name': 'caddy',
  'host': 'localhost',
  'port': 5001,
  'view': ''},
 'client': {'name': 'httpx'},
 'server_details': {'machine_id': 'C02DR0MZQ6LT',
  'cpu_info': {'python_version': '3.10.4.final.0 (64 bit)',
   'cpuinfo_version': [8, 0, 0],
   'cpuinfo_version_string': '8.0.0',
   'arch': 'ARM_8',
   'bits': 64,
   'count': 8,
   'arch_string_raw': 'arm64',
   'brand_raw': 'Apple M1'}},
 'client_details': {'machine_id': 'C02DR0MZQ6LT',
  'cpu_info': {'python_version': '3.10.4.final.0 (64 bit)',
   'cpuinfo_version': [8, 0, 0],
   'cpuinfo_version_string': '8.0.0',
   'arch': 'ARM_8',
   'bits': 64,
   'count': 8,
   'arch_string_raw': 'arm64',
   'brand_raw': 'Apple M1'}},
 'file_size': 100000,
 'elapsed': None,
 'complete_size': 375000000}

<IPython.core.display.Javascript object>

In [ ]:
# dont_test

result.elapsed = client_control_client.measure(benchmark_client, epoch)

<IPython.core.display.Javascript object>

In [ ]:
# dont_test

repository = SqliteRepository.build_repository("results.db")
register_default_tables(repository)
results = repository.tables["result"]

<IPython.core.display.Javascript object>

In [ ]:
# dont_test

result_id = results.add_result(result)

<IPython.core.display.Javascript object>

In [ ]:
# dont_test

results_from_database = results.get_results()

cols = set(["server", "client", "elapsed", "file_size_h", "bytes_per_second_h"])
pd.DataFrame(
    [
        {k: v for k, v in r.dict_for_pandas().items() if k in cols}
        for r in results_from_database
    ]
)

,server,client,elapsed,file_size_h,bytes_per_second_h
0,fastAPI/uvicorn,wrk,2.760704,97.66KB,129.54MB
1,fastAPI/uvicorn,httpx,5.082084,97.66KB,70.37MB
2,caddy,httpx,4.367089,97.66KB,81.89MB
3,caddy,wrk,0.150420,97.66KB,2.32GB
4,caddy,httpx,4.407811,97.66KB,81.14MB


<IPython.core.display.Javascript object>

In [ ]:
# dont_test

# byte = 8
# gigabit = 10 ** 9
# bandwidth = gigabit / byte
# 
# #file_sizes = [10 ** 7, 10 ** 6]
# file_sizes = [10 ** 7, 10 ** 6, 10 ** 5]
# # file_sizes = [10 ** 7]
# 
# benchmark = Benchmark(
#     bandwidth=bandwidth,
#     duration=3,
#     file_sizes=file_sizes,
#     # servers=[NginxDockerServer(), FastAPIUvicornServer()],
#     # servers=[FastAPIUvicornServer()],
#     servers = [DjangoGunicornWSGIServer()],
#     #clients=[HttpxClient(name="httpx"), AioHttpClient(name="aiohttp"), WrkClient(name="wrk")],
#     clients=[AioHttpClient(name="aiohttp"),],
#     repository=SqliteRepository.build_repository("results.db"),
# )
# benchmark.create_rows()

In [ ]:
# dont_test

benchmark.run()

In [ ]:
# dont_test

benchmark.results_frame

In [ ]:
# dont_test

benchmark.results_frame

In [ ]:
# dont_test

benchmark.results_frame

,server,client,file_size,elapsed,complete_size,file_size_h,bytes_per_second,bytes_per_second_h
0,fastAPI/uvicorn,httpx,10000000,5.920403,375000000,9.54MB,6.334028e+07,60.41MB
1,fastAPI/uvicorn,httpx,1000000,5.825266,375000000,976.56KB,6.437474e+07,61.39MB
2,fastAPI/uvicorn,httpx,100000,10.815824,375000000,97.66KB,3.467142e+07,33.07MB


In [ ]:
# dont_test

benchmark.results_frame

,server,client,file_size,elapsed,complete_size,file_size_h,bytes_per_second,bytes_per_second_h
0,fastAPI/uvicorn,httpx,10000000,5.920403,375000000,9.54MB,6.334028e+07,60.41MB
1,fastAPI/uvicorn,httpx,1000000,5.825266,375000000,976.56KB,6.437474e+07,61.39MB
2,fastAPI/uvicorn,httpx,100000,10.815824,375000000,97.66KB,3.467142e+07,33.07MB


In [ ]:
# dont_test

benchmark.results_frame

,server,client,file_size,elapsed,complete_size,file_size_h,bytes_per_second,bytes_per_second_h
0,fastAPI/uvicorn,httpx,10000000,5.920403,375000000,9.54MB,6.334028e+07,60.41MB
1,fastAPI/uvicorn,httpx,1000000,5.825266,375000000,976.56KB,6.437474e+07,61.39MB
2,fastAPI/uvicorn,httpx,100000,10.815824,375000000,97.66KB,3.467142e+07,33.07MB


In [ ]:
# dont_test

benchmark.results_frame

,server,client,file_size,elapsed,complete_size,platform,file_size_h,bytes_per_second,bytes_per_second_h
0,fastAPI/uvicorn,httpx,10000000,6.037067,375000000,x86_64,9.54MB,6.211625e+07,59.24MB
1,fastAPI/uvicorn,httpx,1000000,6.055068,375000000,x86_64,976.56KB,6.193159e+07,59.06MB
2,fastAPI/uvicorn,httpx,100000,29.964979,375000000,x86_64,97.66KB,1.251461e+07,11.93MB
3,fastAPI/uvicorn,aiohttp,10000000,5.628240,375000000,x86_64,9.54MB,6.662829e+07,63.54MB
4,fastAPI/uvicorn,aiohttp,1000000,5.327919,375000000,x86_64,976.56KB,7.038395e+07,67.12MB
5,fastAPI/uvicorn,aiohttp,100000,8.444319,375000000,x86_64,97.66KB,4.440855e+07,42.35MB
6,fastAPI/uvicorn,wrk,10000000,2.772339,375000000,x86_64,9.54MB,1.352648e+08,129.0MB
7,fastAPI/uvicorn,wrk,1000000,7.399281,375000000,x86_64,976.56KB,5.068060e+07,48.33MB
8,fastAPI/uvicorn,wrk,100000,7.345828,375000000,x86_64,97.66KB,5.104939e+07,48.68MB


In [ ]:
# dont_test

benchmark.results_frame

,server,client,file_size,elapsed,complete_size,file_size_h,bytes_per_second,bytes_per_second_h
0,fastAPI/uvicorn,httpx,10000000,6.212713,375000000,9.54MB,6.036010e+07,57.56MB
1,fastAPI/uvicorn,httpx,1000000,7.464866,375000000,976.56KB,5.023533e+07,47.91MB
2,fastAPI/uvicorn,httpx,100000,30.550003,375000000,97.66KB,1.227496e+07,11.71MB
3,fastAPI/uvicorn,aiohttp,10000000,6.143259,375000000,9.54MB,6.104252e+07,58.21MB
4,fastAPI/uvicorn,aiohttp,1000000,5.936163,375000000,976.56KB,6.317212e+07,60.25MB
5,fastAPI/uvicorn,aiohttp,100000,9.707379,375000000,97.66KB,3.863041e+07,36.84MB


In [ ]:
# dont_test

benchmark.results_frame

,server,client,file_size,elapsed,complete_size,file_size_h,bytes_per_second,bytes_per_second_h
0,fastAPI/uvicorn,httpx,10000000,7.621326,375000000,9.54MB,4.920404e+07,46.92MB
1,fastAPI/uvicorn,httpx,1000000,8.129900,375000000,976.56KB,4.612603e+07,43.99MB
2,fastAPI/uvicorn,httpx,100000,31.303474,375000000,97.66KB,1.197950e+07,11.42MB
3,fastAPI/uvicorn,aiohttp,10000000,8.092180,375000000,9.54MB,4.634103e+07,44.19MB
4,fastAPI/uvicorn,aiohttp,1000000,6.438935,375000000,976.56KB,5.823944e+07,55.54MB
5,fastAPI/uvicorn,aiohttp,100000,9.158107,375000000,97.66KB,4.094732e+07,39.05MB


In [ ]:
# dont_test

benchmark.results_frame

,server,client,file_size,elapsed,complete_size,file_size_h,bytes_per_second,bytes_per_second_h
0,fastAPI/uvicorn,httpx,10000000,3.325116,375000000,9.54MB,1.127780e+08,107.55MB
1,fastAPI/uvicorn,httpx,1000000,3.553887,375000000,976.56KB,1.055183e+08,100.63MB
2,fastAPI/uvicorn,httpx,100000,6.097791,375000000,97.66KB,6.149768e+07,58.65MB
3,fastAPI/uvicorn,aiohttp,10000000,3.505289,375000000,9.54MB,1.069812e+08,102.03MB
4,fastAPI/uvicorn,aiohttp,1000000,3.478566,375000000,976.56KB,1.078030e+08,102.81MB
5,fastAPI/uvicorn,aiohttp,100000,4.947546,375000000,97.66KB,7.579515e+07,72.28MB
